In [3]:
import requests
import datetime
import pandas as pd
from bs4 import BeautifulSoup


# Scraping the website
now = datetime.datetime.now()

# Using str now vals for month and year in the link
url = 'https://www.islamicfinder.org/prayer-times/printmonthlyprayer/?timeInterval=month&month='+str(now.month-1)+'&year='+str(now.year)+'&calendarType=Gregorian'

response = requests.get(url)
soup = BeautifulSoup(response.content, 'html.parser')

# Extract data from table
table = soup.find("table", id="monthly-prayers") 
data = []
rows = table.find_all(class_=['row-body','prayer-row-title row-title'])
for i,row in enumerate(rows):
    cols = row.find_all('td')
    cols = [col.text for col in cols]
    if i == 0:
        headers = cols
    else:
        data.append(cols)

# Create DataFrame
df = pd.DataFrame(data,columns=headers)


#df = df.drop("Rajab", axis=1)
# Drop the second column
df = df.drop(df.columns[1], axis=1)

df = df.drop("Day", axis=1)


df = df.rename(columns={df.columns[0]: "Day"})

# Determine the month and year with current date and time in string format



df["Month"] = str(now.month)
df["Year"] = str(now.year)

#merge day and month and year
df['Date'] = df['Month'] + '/' + df['Day'] + '/' + df['Year']

#convert Date to datetime
df['Date'] = pd.to_datetime(df['Date'], format='%m/%d/%Y')

df.head()

,Day,Fajr,Sunrise,Dhuhr,Asr,Maghrib,Isha,Month,Year,Date
0,01,05:44 AM,07:03 AM,12:05 PM,02:44 PM,05:06 PM,06:26 PM,2,2023,2023-02-01
1,02,05:43 AM,07:02 AM,12:05 PM,02:45 PM,05:07 PM,06:27 PM,2,2023,2023-02-02
2,03,05:42 AM,07:01 AM,12:05 PM,02:46 PM,05:09 PM,06:28 PM,2,2023,2023-02-03
3,04,05:41 AM,07:00 AM,12:05 PM,02:47 PM,05:10 PM,06:29 PM,2,2023,2023-02-04
4,05,05:40 AM,06:58 AM,12:05 PM,02:48 PM,05:11 PM,06:30 PM,2,2023,2023-02-05


In [7]:
# expand each row into 6 rows based on column names (Fajr, Sunrise, Dhuhr, Asr, Maghrib, Isha) and convert to one single columnn of time

namazdf = pd.melt(df, id_vars=['Date'], value_vars=['Fajr','Dhuhr','Asr','Maghrib','Isha'], var_name='Namaz', value_name='Namaz Time')

# merge namaztime and date columns
namazdf['Namaz Time'] = namazdf['Date'].astype(str) + ' ' + namazdf['Namaz Time'].astype(str)

# convert to datetime
namazdf['Namaz Time'] = pd.to_datetime(namazdf['Namaz Time'], format='%Y-%m-%d %I:%M %p')

# convert namaztime from CST to UTC
namazdf['Namaz Time'] = namazdf['Namaz Time'].dt.tz_localize('US/Central').dt.tz_convert('UTC')

namazdf = namazdf[["Namaz", "Namaz Time"]]
namazdf[namazdf["Namaz"] == "Maghrib"]


,Namaz,Namaz Time
84,Maghrib,2023-02-01 23:06:00+00:00
85,Maghrib,2023-02-02 23:07:00+00:00
86,Maghrib,2023-02-03 23:09:00+00:00
87,Maghrib,2023-02-04 23:10:00+00:00
88,Maghrib,2023-02-05 23:11:00+00:00
89,Maghrib,2023-02-06 23:13:00+00:00
90,Maghrib,2023-02-07 23:14:00+00:00
91,Maghrib,2023-02-08 23:15:00+00:00
92,Maghrib,2023-02-09 23:16:00+00:00
93,Maghrib,2023-02-10 23:18:00+00:00


In [8]:
# Export to CSV
namazdf.to_csv('prayer_times3.0.csv', index=False)

In [9]:
#convert to ics and set timezone to GMT-6
from ics import Calendar, Event
from datetime import datetime, timedelta

c = Calendar()



for index, row in namazdf.iterrows():
    e = Event()
    e.name = row["Namaz"]
    #set dtstart and dtend to the same time
    e.begin = row["Namaz Time"]
    e.end = row["Namaz Time"]
    c.events.add(e)

with open('namaz2.ics', 'w') as my_file:
    my_file.writelines(c)

